In [29]:
from nes_py.wrappers import JoypadSpace
import gym_tetris
from gym_tetris.actions import MOVEMENT

from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [37]:
# The nn

class Net(nn.Module):
    def __init__(self,obs_size,hidden_size,n_actions):
        super(Net,self).__init__()
        self.net = nn.Sequential(
        nn.Linear(obs_size,hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size,n_actions)
        )
        
    def adjust_observations(obs):
        wab= [sum(b)/3 for a in obs for b in a]
        return wab 
    
    def forward(self,x):
        x = adjust_observations(x)
        return self.net(x)
    
   

In [31]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70
Episode =namedtuple('Episode',field_names=['reward','steps'])# like a struct
EpisodeStep=namedtuple('EpisodeStep',field_names=['observation','action'])


In [32]:
def iterate_batches(env,net,batch_size):
    episode_reward=0
    batch=[]
    episode_steps=[]
    obs = env.reset()
    sm=nn.Softmax(dim=1)
    while True:
        obs_v=torch.FloatTensor([obs])
        actions_probs=sm(net(obs_v)).data.numpy()[0]
        action_chosen=np.random.choice(len(actions_probs),p=actions_probs)
        next_obs, reward, is_done, _ = env.step(action_chosen)
        ep_step=EpisodeStep(observation=obs,action=action_chosen)
        episode_reward+=reward
        episode_steps.append(ep_step)
        
        if is_done:
            batch.append(Episode(reward=episode_reward,steps=episode_steps))
            next_obs=env.reset()
            episode_steps=[]
            episode_reward=0
            
        if len(batch)==batch_size:
            yield batch
            batch=[]
        obs = next_obs

In [33]:
def filter_batch(batch,percentile):
    rewards = list(map(lambda s :s.reward,batch))
    reward_boundary= np.percentile(rewards,percentile)
    reward_mean= np.mean(rewards)
    train_acts=[]
    train_obs=[]
    for episode in batch:
        if episode.reward <reward_boundary:
            continue
        train_act.extend(map(lambda es:es.action,episode.steps))
        train_obs.extend(map(lambda es:es.observation,episode.steps))
    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_boundary, reward_mean

In [41]:
torch.cuda.is_available()

False

In [35]:
if __name__is_available"__main__":
    env = gym_tetris.make('TetrisA-v0')
    env = JoypadSpace(env, MOVEMENT)
    obs_size=env.observation_space.shape[0]*env.observation_space.shape[1]
    net=Net(obs_size,10000,env.action_space.n)
    loss_fn=nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(params=net.parameters(),lr=0.01)
    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))

NameError: name 'adjust_observations' is not defined

In [ ]:


done = True
for step in range(5000):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    env.render()

env.close()